In [ ]:
"""
因子優化模組使用範例
"""

from alanq.optimization import ParameterSpace, FactorOptimizer
from alanq.factors.timing import BreakoutBuyFactor, BreakdownSellFactor, AtrStopSellFactor
from alanq.data import StockDataManager

def example_optimization():
    """範例：單股票因子優化"""
    
    # 1. 準備資料
    print("正在下載資料...")
    data_manager = StockDataManager(["TSLA"], start_date="2020-01-01")
    df = data_manager.get_kl_pd("TSLA")
    
    # 2. 定義參數空間
    print("定義參數空間...")
    param_space = ParameterSpace()
    
    # 添加買入因子
    param_space.add_buy_factor(
        BreakoutBuyFactor,
        {'xd': [20, 30, 40, 50, 60], 'skip_days': [10, 15, 20]}
    )
    
    # 添加賣出因子
    param_space.add_sell_factor(
        BreakdownSellFactor,
        {'xd': [10, 15, 20, 25]}
    )
    
    param_space.add_sell_factor(
        AtrStopSellFactor,
        {'atr_n': [1.0, 1.5, 2.0, 2.5]}
    )
    
    # 查看總組合數
    total_combos = param_space.get_total_combinations()
    print(f"總組合數: {total_combos}")
    
    # 3. 執行優化
    print("\n開始優化...")
    optimizer = FactorOptimizer(
        df=df,
        parameter_space=param_space,
        initial_capital=1_000_000,
        show_progress=True
    )
    
    best_config, results_df = optimizer.optimize()
    
    # 4. 查看結果
    if best_config:
        print(f"\n{'='*60}")
        print("最佳配置:")
        print(f"{'='*60}")
        print(f"買入因子: {best_config['buy_factors']}")
        print(f"賣出因子: {best_config['sell_factors']}")
        print(f"總得分: {best_config['總得分']:.2f}")
        
        print(f"\n{'='*60}")
        print("前 10 名結果:")
        print(f"{'='*60}")
        top_results = optimizer.get_top_n(10)
        display_cols = ['組合編號', '總得分', '策略_總報酬率', '策略_Sharpe', 
                       '策略_最大回撤', '勝率', '盈虧比']
        available_cols = [col for col in display_cols if col in top_results.columns]
        print(top_results[available_cols].to_string())
        
        # 5. 儲存結果
        optimizer.save_results('optimization_results.csv')
    else:
        print("優化失敗，沒有找到有效結果")
    
    return best_config, results_df

if __name__ == "__main__":
    best_config, results_df = example_optimization()